## 3. Implementando os Simuladores para o Caso Exponencial

Cenário $\lambda = 1$ e $\mu  = 2$

Descrição: Considere uma fila M/M/1, em que chegadas ocorrem segundo um fluxo Poisson com taxa 1 chegadas/segundo e serviços são tais que cada serviço é distribuído exponencialmente com média 1/2 segundo. Implemente dois simuladores.

In [1]:
import random

In [3]:
simulation_time = 0
max_steps = 1000
arrival_rate = 1  # (lambda)
service_rate = 2  # (mu)
queue_length = 0

total_arrivals = 0
total_departures = 0
total_waiting_time = 0

arrivals_queue = []

for i in range(max_steps):
    sample_arrival = random.expovariate(arrival_rate)
    sample_departure = random.expovariate(service_rate)

    if queue_length == 0 or sample_arrival < sample_departure:
        # Arrival
        print("Arrival")
        simulation_time += sample_arrival
        total_arrivals += 1
        queue_length += 1
        if queue_length > 1:
            arrivals_queue.append(simulation_time)
    else:
        # Departure
        print("Departure")
        simulation_time += sample_departure
        total_departures += 1
        queue_length -= 1
        if len(arrivals_queue) > 0:
            total_waiting_time += simulation_time - arrivals_queue.pop(0)
    print("Simulation Time: ", simulation_time)
    print("Arrivals Queue: ", arrivals_queue)
    print("Total Waiting Time: ", total_waiting_time)
    print("-")

average_waiting_time = total_waiting_time / total_departures
print("Average waiting time:", average_waiting_time)

Arrival
Simulation Time:  0.1099675950080188
Arrivals Queue:  []
Total Waiting Time:  0
-
Arrival
Simulation Time:  0.36592936261958653
Arrivals Queue:  [0.36592936261958653]
Total Waiting Time:  0
-
Arrival
Simulation Time:  0.5583269457286568
Arrivals Queue:  [0.36592936261958653, 0.5583269457286568]
Total Waiting Time:  0
-
Arrival
Simulation Time:  0.7124062294483104
Arrivals Queue:  [0.36592936261958653, 0.5583269457286568, 0.7124062294483104]
Total Waiting Time:  0
-
Departure
Simulation Time:  0.9599354918331239
Arrivals Queue:  [0.5583269457286568, 0.7124062294483104]
Total Waiting Time:  0.5940061292135373
-
Departure
Simulation Time:  0.9686916857588871
Arrivals Queue:  [0.7124062294483104]
Total Waiting Time:  1.0043708692437676
-
Departure
Simulation Time:  0.9870389081687662
Arrivals Queue:  []
Total Waiting Time:  1.2790035479642232
-
Arrival
Simulation Time:  1.1387183159532648
Arrivals Queue:  [1.1387183159532648]
Total Waiting Time:  1.2790035479642232
-
Departure
Simu